In [2]:
import requests
import time
import json
import threading

In [4]:
root = 'http://www.leylobby.gob.cl/api/v1'
apikey = "$2y$10$2g/G3Jf9W6gKhR2vvDVSwOv207mF7s/MUjKoaJRFUyT"
headers = {'User-Agent':'Mozilla/5.0','Api-Key':'$2y$10$2g/G3Jf9W6gKhR2vvDVSwOv207mF7s/MUjKoaJRFUyT'}

In [3]:
root_res = requests.get(root,headers=headers)

In [4]:
content = root_res.json()
for c in content:
    print(content[c])

https://www.leylobby.gob.cl/api/v1/cargos-activos
https://www.leylobby.gob.cl/api/v1/cargos-pasivos
https://www.leylobby.gob.cl/api/v1/audiencias
https://www.leylobby.gob.cl/api/v1/donativos
https://www.leylobby.gob.cl/api/v1/viajes
https://www.leylobby.gob.cl/api/v1/instituciones


In [5]:
def getData(page, psize=10000):
    #make first request to get number of pages
    fails = []
    start = time.time()
    fname = page.split('/')[-1]
    print('Getting ',fname)
    result = requests.get(page+'?per_page='+str(psize), headers = headers)
    res_json = result.json()
    npages = res_json['last_page']
    print('Number of resuts:',res_json['total'])
    print('At',psize,'results per page, number of pages:',npages)
    #init results
    data = []
    data = res_json['data']
    #make request for each remaining page
    for i in range(2, npages+1):
        t = (time.time()-start)/60
        print('Getting page',i,'/',npages,'of',fname,'at',"{:.2f}".format(t),'min since start',end='\r', flush = True)
        result = requests.get(page+'?per_page='+str(psize)+'&page='+str(i), headers = headers)
        #save results
        try:
            data = data + result.json()['data']
        except:
            fails.append(i)
            print('')
            print('page', i,'of',fname, 'failed')
    #store results
    print('')
    print('Storing to file:',fname,'.json')
    file = open('Lobby/'+fname+'.json','w')
    json.dump(data,file)
    file.close()
    t = (time.time()-start)/60
    print('')
    print(fname,'done in:',"{:.2f}".format(t),'min')
    print(len(fails), 'pages failed with page size:', psize, '\n', fails)

In [ ]:

pages=[10000,1000,10000,10000,10000,10000]
i=0
for c in content:
    start = time.time()
    getData(content[c],pages[i])
    #th = threading.Thread(target = getData, args = (content[c],pages[i]))
    #th.start()
    i+=1

t = (time.time()-start)/60    
print('All done in',"{:.2f}".format(t),'min')


Getting  cargos-activos
Number of resuts: 255741
At 10000 results per page, number of pages: 26
Getting page 26 / 26 of cargos-activos at 18.31 min since start
Storing to file: cargos-activos .json

cargos-activos done in: 19.05 min
0 pages failed with page size: 10000 
 []
Getting  cargos-pasivos
Number of resuts: 246219
At 1000 results per page, number of pages: 247
Getting page 16 / 247 of cargos-pasivos at 0.50 min since start
page 16 of cargos-pasivos failed
Getting page 19 / 247 of cargos-pasivos at 0.59 min since start
page 19 of cargos-pasivos failed
Getting page 22 / 247 of cargos-pasivos at 0.68 min since start
page 22 of cargos-pasivos failed
Getting page 23 / 247 of cargos-pasivos at 0.70 min since start
page 23 of cargos-pasivos failed
Getting page 25 / 247 of cargos-pasivos at 0.75 min since start
page 25 of cargos-pasivos failed
Getting page 26 / 247 of cargos-pasivos at 0.77 min since start
page 26 of cargos-pasivos failed
Getting page 247 / 247 of cargos-pasivos at 8.6

In [7]:
getData('https://www.leylobby.gob.cl/api/v1/viajes',10000)
getData('https://www.leylobby.gob.cl/api/v1/instituciones',10000)

Getting  viajes
Number of resuts: 346104
At 10000 results per page, number of pages: 35
Getting page 35 / 35 of viajes at 7.22 min since start
Storing to file: viajes .json

viajes done in: 7.75 min
0 pages failed with page size: 10000 
 []
Getting  instituciones
Number of resuts: 689
At 10000 results per page, number of pages: 1

Storing to file: instituciones .json

instituciones done in: 0.03 min
0 pages failed with page size: 10000 
 []


In [37]:
#Terminar e recolectar cargos pasivos
cp_base = "https://www.leylobby.gob.cl/api/v1/cargos-pasivos"
def getFailed(pnum, psize, fails):
    data = []
    for i in range(psize*(pnum-1)+1,psize*pnum+1):
        result = requests.get(cp_base+'?per_page=1&page='+str(i), headers = headers)
        try:
            #print("Entry",i,"retrieved")
            data += result.json()['data']
        except:
            print("Entry",i,"cannot be retrieved")
            fails.append(i)
    return data
    
    
#    if size == 1:
#        result = requests.get(cp_base+'?per_page='+str(psize)+'&page='+str(pnumb), headers = headers)
#        try:
#            data = data +result.json()['data']
#        except:
#            print("Entry number",pnumb,"cannot be retrieved")
#    else:
#        if psize%2 == 0:
#            new_psize1 = new_psize2 = psize/2
#            new_pnum1 = pnumb*2
#            new_pnum2 = new_pnumb -1
#        else:
#            new_psize1 = psize//2
#            new_psize2 = new_psize1 + 1
            

In [38]:
pnums = [16, 19, 22, 23, 25, 26]
fails = []
data = []

for i in pnums:
    s = time.time()
    data += getFailed(i,1000,fails)
print("Elapsed time:",time.time()-s)

Entry 15054 cannot be retrieved
Entry 18917 cannot be retrieved
Entry 18922 cannot be retrieved
Entry 21138 cannot be retrieved
Entry 22320 cannot be retrieved
Entry 24148 cannot be retrieved
Entry 24154 cannot be retrieved
Entry 25074 cannot be retrieved
Elapsed time: 1019.5776851177216


In [41]:
file = open('Lobby/r_cargos-pasivos.json','w')
json.dump(data,file)
file.close()

In [39]:
print(fails)

[15054, 18917, 18922, 21138, 22320, 24148, 24154, 25074]


In [11]:
#test
data = []
result = requests.get("https://www.leylobby.gob.cl/api/v1/donativos/125", headers = headers)
print(result.json())
data+=[result.json()]

{'id_donativo': '125', 'id_cargo_pasivo': '219', 'nombres': 'Ximena Cecilia ', 'apellidos': 'Rincon Gonzalez', 'pais': 'CHL', 'cargo': 'Ministro(a)', 'ocasion': 'Regalo Navidad ', 'descripcion': 'Publicación del Museo Histórico Nacional  y tarjeta navideña', 'fecha': '2014-12-18', 'sujeto_pasivo_url': 'https://www.leylobby.gob.cl/api/v1/cargos-pasivos/316', 'institucion_url': 'https://www.leylobby.gob.cl/api/v1/instituciones/157', 'institucion': {'codigo': 'AF001', 'nombre': 'Subsecretaría General de la Presidencia'}, 'donantes': [{'id_donativo': '125', 'nombre': 'Museo Histórico Nacional', 'pais': 'CHL', 'tipo': 'persona_juridica', 'giro': None, 'actividades': '', 'domicilio': None, 'representante_legal': None, 'naturaleza': None, 'directorio': None}]}


In [9]:
print(data)

[{'id_donativo': '125', 'id_cargo_pasivo': '219', 'nombres': 'Ximena Cecilia ', 'apellidos': 'Rincon Gonzalez', 'pais': 'CHL', 'cargo': 'Ministro(a)', 'ocasion': 'Regalo Navidad ', 'descripcion': 'Publicación del Museo Histórico Nacional  y tarjeta navideña', 'fecha': '2014-12-18', 'sujeto_pasivo_url': 'https://www.leylobby.gob.cl/api/v1/cargos-pasivos/316', 'institucion_url': 'https://www.leylobby.gob.cl/api/v1/instituciones/157', 'institucion': {'codigo': 'AF001', 'nombre': 'Subsecretaría General de la Presidencia'}, 'donantes': [{'id_donativo': '125', 'nombre': 'Museo Histórico Nacional', 'pais': 'CHL', 'tipo': 'persona_juridica', 'giro': None, 'actividades': '', 'domicilio': None, 'representante_legal': None, 'naturaleza': None, 'directorio': None}]}]


De los resultados obtenidos, faltan detalles de audiencias, viajes y donativos.
* Audiencias se pueden obtener directamente
* Viajes y donativos tienen ids incorrectas en la url
    * Se puede sacar todos los urls hasta tener todos los donativos (no existen todos los ids como url)

In [ ]:
#Obtención de audiencias
daud_url = "https://www.leylobby.gob.cl/api/v1/audiencias/"

In [ ]:
#Getting Donativos by url - takes around 10h
ddon_url = "https://www.leylobby.gob.cl/api/v1/donativos/"

total = 0
invalid = 0
i = 0
data = []
start = time.time()
while total < 31069:
    result = requests.get(ddon_url+str(i), headers = headers)
    try:
        total+=1
        data+=[result.json()]
    except:
        invalid+=1
        total-=1
    i+=1
    if((time.time()-start)>36000):
        break
tot_time = time.time()-start
print(tot_time)